# Thuật toán Linear Regression (Hồi quy tuyến tính).
Nội dung:

1. Cài đặt thuật toán.
2. Sử dụng thư viện scikit-learn.
3. Tối ưu hàm mất mát sử dụng thuật toán gradient descent.
4. Mở rộng thuật toán hồi quy tuyến tính. (cập nhật)

1 Cài đặt thuật toán.
 Linear regression là thuật toán supervised, quan hệ đầu vào giữa đầu ra được mô tả bởi một hàm tuyến tính.

In [1]:
import numpy as np
import matplotlib.pyplot as plt # thư viện hiển thị dữ liệu
from matplotlib.backends.backend_pdf import PdfPages

In [2]:
def linear_data(X):
    N,d = X.shape
    one = np.ones((N,1))
    X_bar = np.concatenate((one,X),axis = 1)
    return X_bar
def linear_regression(X_train,y_train):
    X_bar = linear_data(X_train)
    A = np.dot(X_bar.T, X_bar)
    b = np.dot(X_bar.T, y_train)
    w = np.dot(np.linalg.pinv(A), b)
    return w
def linearregression_predic(X_train,y_train,X_test):
    X1 = linear_data(X_test)
    w = linear_regression(X_train,y_train)
    return X1.dot(w)
    

Ví dụ 1. Bài toán về chiều cao và cân nặng.


X_train, y_train: lần lượt là chiều cao và cân nặng tương ứng, X_test là chiều cao mới chưa biết cân nặng, y_pred: là cân nặng dự đoán.

In [3]:

X_train = np.array([[147, 150, 153, 158, 163, 165, 168, 170, 173, 175, 178, 180, 183]]).T
y_train = np.array([ 49, 50, 51,  54, 58, 59, 60, 62, 63, 64, 66, 67, 68])
X_test = np.array([[160,152]]).T
y_pred = linearregression_predic(X_train,y_train,X_test)
print(linear_regression(X_train,y_train))
print(y_pred)

[-33.73541021   0.55920496]
[55.7373837  51.26374401]


2 Sử dụng thư viện scikit-learn.


srouce: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html

In [4]:
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split # chia dữ liệu thành hai phần test và set
#dữ liệu
X = np.array([[147, 150, 153, 158, 163, 165, 168, 170, 173, 175, 178, 180, 183, 185, 187]]).T
y = np.array([ 49, 50, 51,  54, 58, 59, 60, 62, 63, 64, 66, 67, 68, 71, 72])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 2 ) # lấy 2 điểm dữ liệu dể dự đoán
print(X_train)
print(y_train)


[[163]
 [147]
 [187]
 [185]
 [178]
 [165]
 [173]
 [158]
 [175]
 [153]
 [170]
 [168]
 [180]]
[58 49 72 71 66 59 63 54 64 51 62 60 67]


In [5]:
regr = linear_model.LinearRegression() # khởi tạo mô hình
regr.fit(X_train,y_train)           # luyện mô hình


print("Tham số của mô hình: ")
print(regr.coef_[0], regr.intercept_)
print("Dữ liệu dự đoán:",X_test)
y_pred = regr.predict(X_test)
print("Giá trị dự đoán",y_pred)
print("Gía trị thực:",y_test)


Tham số của mô hình: 
0.5854614300810146 -37.93738992603031
Dữ liệu dự đoán: [[150]
 [183]]
Giá trị dự đoán [49.88182459 69.20205178]
Gía trị thực: [50 68]


3 Tối ưu hàm mất mát bằng thuật toán gradient descent.
Tìm tham số w thông qua thuật toán gradient descent.

Thuật toán linear regression tìm tham số w bằng cách đạo hàm của hàm mất mát và cho bằng 0. Việc giải phương trình đạo hàm bằng 0 gây khó khăn khi điểm dữ liệu có số chiều lớn, thuật toán gradient descent (GD) đơn giản giúp tìm tham số w gần với nghiệm của bài toán cần tìm (khi đạo hàm xấp xỉ 0). 


VD2. Khởi tạo 500 điểm dữ liệu được chọn gần với đường thẳng y = 1 + 2x. Đưa ra tham số của mô hình.

In [6]:
from sklearn.linear_model import LinearRegression
X = np.random.rand(500) # random ra 500 điểm
y = 1 + 2*X + .2* np.random.rand(500)
#print(X)
#print(y)
model = LinearRegression()
X = X.reshape(-1,1) # reshape về ma trận X gồm N hàng 1 cột
y = y.reshape(-1,1)
model.fit(X,y)
w, b = model.coef_[0][0], model.intercept_[0]
sol_sklearn = np.array([b,w])
print("Tham số w giải bằng sklearn",sol_sklearn)
# print(linear_regression(X,y)): test kết quả với phần 1.

Tham số w giải bằng sklearn [1.09926074 2.0018674 ]


sử dụng GD để tìm tham số w

In [7]:
one = np.ones((X.shape[0],1))
X_bar =np.concatenate((one,X),axis = 1)
# đạo hàm hàm mất mát
def grad(w):
    N = X_bar.shape[0]
    return 1/N*X_bar.T.dot(X_bar.dot(w)-y)

In [8]:
# giá trị hàm mất mát
def cost(w):
    N = X_bar.shape[0]
    return 0.5/N* np.linalg.norm(y-X_bar.dot(w))**2
    

In [9]:
# thuật toán gradient tìm w
def lr_gd(w_init, grad, eta):
    w = [w_init]
    cost_hist =[] # lưu giá trị hàm mất mát sau mỗi vòng lặp
    for it in range(100):
        w_new = w[-1]- eta*grad(w[-1])
        if np.linalg.norm(grad(w_new))/len(w_new) <1e-3:
            break
        cost_hist.append(cost(w_new))
        w.append(w_new)
    return (w, it,cost_hist)


In [10]:
w_init = np.array([[1],[4]])
(w1, it1, cost_hit) =lr_gd(w_init, grad, 1)

print("Kết quả tham số:\n",w1[-1])
print("\nSố vòng lặp",it1)
print("Hàm mất mát:\n",cost_hit)

Kết quả tham số:
 [[1.08443692]
 [2.0292238 ]]

Số vòng lặp 58
Hàm mất mát:
 [0.13297544766505878, 0.08746133697975607, 0.07412299541859518, 0.06449329201412769, 0.056269032101059845, 0.0491302812580024, 0.04292500465677666, 0.03753048212410622, 0.03284073481112168, 0.028763682641915672, 0.025219279376656925, 0.022137936761934675, 0.019459157964983733, 0.017130349799001757, 0.015105790145124861, 0.013345730277600613, 0.011815614466324084, 0.01048540153486755, 0.009328975053960848, 0.008323630590591423, 0.00744962994574705, 0.006689813629030863, 0.006029263961758063, 0.005455012194146739, 0.004955783886355393, 0.004521777554364614, 0.004144472234797653, 0.0038164601905481374, 0.0035313014726794613, 0.0032833974831703145, 0.003067881056129655, 0.0028805208994160123, 0.002717638520537588, 0.002576036005817, 0.0024529332348895206, 0.0023459132978500795, 0.0022528750434092225, 0.0021719918264231012, 0.002101675644876694, 0.002040545962211931, 0.001987402602881067, 0.0019412021889764627, 0.0

4 Kết luận, mở rộng

Thuật toán linear regression nhạy cảm với tín hiệu nhiễu (overfitting).